In [ ]:
try:
    # se estiver no colab vai rodar o bloco abaixo
    from google.colab import data_table, drive
    drive.mount('/content/drive', force_remount=True)

    data_table.enable_dataframe_formatter()
    data_table.DataTable.max_columns = 50
    path = "/content/drive/MyDrive/doutorado/lma" # caminho do google drive
except:
    # se nao estiver no colab vai rodar o bloco abaixo
    path = "." # caminho local

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import gzip
from io import StringIO
from matplotlib import cm, colors
from matplotlib.ticker importFormatStrFormatter, MultipleLocator

camera = {"lat": 39.3384, "lon": -112.70096}


def m2km(x, _): return f'{x/1000:g}'  # formatar plot label; metro para km

In [ ]:
# carrega dados do .csv
dados = pd.read_csv(path + "/lma.csv", sep=";", index_col=0, parse_dates=True, na_values=["//", ""], skipinitialspace=True, skip_blank_lines=True, encoding="unicode_escape")
dados.columns = dados.columns.str.strip()

dados  # visualiza dados do csv para verificar se esta tudo ok

In [ ]:
# filtra o stroke, adiciona a coluna "time" de acordo com os segundos do dia
def stroke_filter(stroke):
    stroke["time"] = stroke.name.hour * 60 * 60 + stroke.name.minute * 60 + stroke.name.second + stroke.name.microsecond/1e6

    print("stroke sendo usado:")
    print(stroke.name.strftime('%Y-%m-%d %H:%M:%S.%f'))
    return stroke

In [ ]:
# obtem os dados de lma no formato .gz. após isso, extrai e processa os dados
def get_lmagz(stroke):
    fname = "%s%02d00" % (stroke.name.strftime('TA_%y%m%d_%H'), int(stroke.name.minute/10)*10)
    url = '%s/lma/talma/lightning.nmt.edu/talma/data/%s/%s_0600.dat.gz' % (path, stroke.name.strftime('%y%m%d'), fname)
    print("URL dos dados de LMA:", url)

    lmagz = gzip.open(url, 'rt').read()
    lmagz = lmagz.split('*** data ***')

    lmagz = pd.read_csv(StringIO(lmagz[1]), sep='\s+')
    lmagz.columns = ["time", "lat", "lon", "alt", "chi", "P", "mask"]
    lmagz.head()
    return lmagz

In [ ]:
# filtra e processa os dados de lma obtidos anteriormente
def lma_filter(lmagz, stroke):
    lma = lmagz.copy()
    lma = lma[(lma.chi <= stroke.chi)]
    lma = lma[(lma.time >= stroke.time - stroke.ti) & (lma.time <= stroke.time + stroke.cc)]
    lma = lma[(lma.lat >= stroke.lat - stroke.ds) & (lma.lat <= stroke.lat + stroke.ds)]
    lma = lma[(lma.lon >= stroke.lon - stroke.ds) & (lma.lon <= stroke.lon + stroke.ds)]
    lma = lma[(lma.alt >= stroke.hi) & (lma.alt <= stroke.hf)]

    lma["dt"] = lma.time - stroke.time
    lma = lma.sort_values(by=['time'])

    lma_neg = lma[(lma["dt"] <= 0)]  # valores antes do stroke
    lma_pos = lma[(lma["dt"] >= 0)]  # valores depois do stroke

    print("stroke time:", stroke.time)
    print("%-6s %6s %6s %6s" % ("pontos", "total", "acima", "abaixo"))
    print("%-6s %6d %6d %6d" % ("total",     lma.time.size,     lma[(lma.alt >= stroke.hleader)].time.size,         lma[(lma.alt <= stroke.hleader)].time.size))
    print("%-6s %6d %6d %6d" % ("antes", lma_neg.time.size, lma_neg[(lma_neg.alt >= stroke.hleader)].time.size, lma_neg[(lma_neg.alt <= stroke.hleader)].time.size))
    print("%-6s %6d %6d %6d" % ("após ", lma_pos.time.size, lma_pos[(lma_pos.alt >= stroke.hleader)].time.size, lma_pos[(lma_pos.alt <= stroke.hleader)].time.size))
    print("\nmin", lma.min())
    print("\nmax", lma.max())
    return lma

In [ ]:
# gera o plot dos dados e salva em .png
def plot_lma(lma, stroke):
    ax = {}
    im = {}

    cmap = colors.LinearSegmentedColormap.from_list('r2b', ['blue', 'red'])
    cmap = colors.ListedColormap(["blue", "red"])
    cmap = "jet"

    fig = plt.figure()
    fig.set_size_inches(10, 10)
    grid = (15, 15)

    # xoo plot
    # ooo superior
    # ooo histograma de alt
    p = (0, 0)
    ax[p] = plt.subplot2grid(grid, p, rowspan=2, colspan=2)
    ax[p] = lma.alt.plot.hist(bins=range(0, 15000, 100), orientation='horizontal', color='black')
    ax[p].xaxis.tick_top()
    ax[p].xaxis.set_label_position('top')
    ax[p].yaxis.set_major_locator(MultipleLocator(base=3000))
    # ax[p].xaxis.set_major_locator(MultipleLocator(base=50))
    ax[p].yaxis.set_major_formatter(m2km)
    ax[p].set(xlabel="Frequência", ylabel="Alt (km)", ylim=(stroke.hi, stroke.hf))

    # oxx plot
    # ooo superior
    # ooo lon x alt
    p = (0, 2)
    ax[p] = plt.subplot2grid(grid, p, rowspan=2, colspan=8)
    ax[p].yaxis.tick_right()
    ax[p].yaxis.set_label_position("right")
    ax[p].set(ylabel="Alt (km)", xlabel="Lon (°)", xlim=(stroke.lon - stroke.ds, stroke.lon + stroke.ds), ylim=(stroke.hi, stroke.hf))
    ax[p].xaxis.tick_top()
    ax[p].xaxis.set_label_position('top')
    ax[p].locator_params(nbins=4)
    ax[p].yaxis.set_major_locator(MultipleLocator(base=3000))
    ax[p].yaxis.set_major_formatter(m2km)
    ax[p].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    im[p] = ax[p].scatter(lma.lon, lma.alt, c=lma["dt"], cmap=cmap, norm=colors.TwoSlopeNorm(0), s=3, alpha=0.8, marker=".")
    im[p] = ax[p].scatter(stroke.lon, 250, c="black", edgecolor="white", s=50, alpha=1, marker="v")  # posição do stroke
    im[p] = ax[p].scatter(camera['lon'], 250, c="black", edgecolor="white", s=40, alpha=1, marker="s")  # coord camera

    # ooo plot
    # xoo esquerda
    # ooo lat x alt
    p = (2, 0)
    ax[p] = plt.subplot2grid(grid, p, colspan=2, rowspan=8)
    ax[p].xaxis.tick_bottom()
    ax[p].xaxis.set_label_position("bottom")
    ax[p].set(xlabel="Alt (km)", ylabel="Lat (°)", xlim=(stroke.hi, stroke.hf), ylim=(stroke.lat - stroke.ds, stroke.lat + stroke.ds))
    ax[p].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[p].locator_params(nbins=6)
    ax[p].xaxis.set_major_locator(MultipleLocator(base=3000))
    ax[p].invert_xaxis()
    ax[p].xaxis.set_major_formatter(m2km)
    im[p] = ax[p].scatter(lma.alt, lma.lat, c=lma["dt"], cmap=cmap, norm=colors.TwoSlopeNorm(0), s=1, alpha=0.8, marker=".")
    im[p] = ax[p].scatter(500, stroke.lat, c="black", edgecolor="white", s=50, alpha=1, marker="v")  # posição do stroke
    im[p] = ax[p].scatter(500, camera['lat'], c="black", edgecolor="white", s=40, alpha=1, marker="s")  # coord camera

    # ooo plot
    # oxx central
    # ooo lon x lat
    p = (2, 2)
    ax[p] = plt.subplot2grid(grid, p, colspan=8, rowspan=8)
    ax[p].set(xlabel="Lon (°)", ylabel="Lat (°)", xlim=(stroke.lon - stroke.ds, stroke.lon + stroke.ds), ylim=(stroke.lat - stroke.ds, stroke.lat + stroke.ds))
    ax[p].yaxis.tick_right()
    ax[p].yaxis.set_label_position("right")
    ax[p].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[p].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[p].locator_params(nbins=4, axis='x')
    ax[p].locator_params(nbins=6, axis='y')
    im[p] = ax[p].scatter(lma.lon, lma.lat, c=lma["dt"], cmap=cmap, norm=colors.TwoSlopeNorm(0), s=3, alpha=0.8, marker=".")
    im[p] = ax[p].scatter(stroke.lon, stroke.lat, c="black", edgecolor="white", s=50, alpha=1, marker="v")  # posição do stroke
    im[p] = ax[p].scatter(camera['lon'], camera['lat'], c="black", edgecolor="white", s=40, alpha=1, marker="s")  # coord camera

    # ooo plot
    # ooo inferior
    # xxx alt x t
    p = (11, 0)
    ax[p] = plt.subplot2grid(grid, p, rowspan=2, colspan=10)
    ax[p].set(xlabel="t (ms)", ylabel="Alt (km)", ylim=(stroke.hi, stroke.hf))
    ax[p].yaxis.set_major_locator(MultipleLocator(base=3000))
    ax[p].yaxis.set_major_formatter(m2km)
    ax[p].set_xmargin(0.01)
    ax[p].tick_params(left=True, labelleft=True, bottom=False, labelbottom=False)

    ax[p].annotate('DR1', xy=(0, 0), xytext=(-14, 6), textcoords='offset pixels', color='black', rotation=90)
    ax[p].annotate('DR2', xy=(stroke['subseq'] * 1000, 0), xytext=(-14, 6), textcoords='offset pixels', color='black', rotation=90)
    ax[p].annotate('CC', xy=(stroke['cc_duration'] * 1000, 0), xytext=(-12, 80), textcoords='offset pixels', color='black', rotation=90)

    ax_twin = ax[p].twinx()  # clona eixo x, para mostrar eixo y na direita
    ax_twin.set_yticks(ax[p].get_yticks())
    ax_twin.set_yticklabels(ax[p].get_yticklabels())
    ax_twin.set_ylabel(ax[p].get_ylabel())
    ax_twin.set_ylim(ax[p].get_ylim())

    im[p] = ax[p].scatter(lma["dt"]*1000, lma.alt, c=lma["dt"], cmap=cmap, norm=colors.TwoSlopeNorm(0), s=3, alpha=0.8, marker=".")
    im[p] = ax[p].axvline(x=0, color='orange', ls='dashed', alpha=0.8)
    im[p] = ax[p].axvline(x=stroke['cc_duration'] * 1000, color='grey', ls='dashed', alpha=0.8)
    im[p] = ax[p].axvline(x=stroke['subseq'] * 1000, color='orange', ls='dashed', alpha=0.8)

    # colorbar
    cb = plt.subplot2grid((30, 15), (26, 0), colspan=10)
    map = cm.ScalarMappable(cmap=cmap, norm=colors.TwoSlopeNorm(vmin=(lma["dt"]*1000).min(), vcenter=0, vmax=(lma["dt"]*1000).max()))
    cbar = fig.colorbar(map, cax=cb, label="t (ms)", orientation="horizontal", pad=0.5, fraction=0.07, anchor=(1.0, 0.0))
    cbar.ax.set_xscale('linear')
    cbar.ax.tick_params(top=True, labeltop=False, bottom=True, labelbottom=True)

    txt = f"""Linha {stroke['linha']:.0f}, Multiplicidade {stroke['mult']:.0f}, Ip {stroke['imax']:.1f} kA, CC {stroke['cc_duration'] * 1000:.0f} ms, {stroke.name.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}"""
    print(txt)

    fig.show()
    fig.savefig('%s/%s.png' % (path, txt), bbox_inches='tight', dpi=fig.dpi)

In [ ]:
# roda as funções para cada linha da tabela "dados"
for index, row in dados.iterrows():
    try:
        # executa para cada linha
        stroke = stroke_filter(row)
        lmagz = get_lmagz(stroke)
        lma = lma_filter(lmagz, stroke)
        plot_lma(lma, stroke)
    except:
        # se os dados não existem, passa para a próxima linha, sem parar o código
        pass